In [2]:
#!pip install tensorflow-recommenders

In [1]:
import json
import numpy as np
import pandas as pd
from typing import Dict, Text

import tensorflow as tf
import tensorflow_recommenders as tfrs

In [2]:
df = pd.read_csv('../Notebook files/Digital_Music.csv')

df = df[:500]
df.head()

,0001388703,A1ZCPG3D3HGRSS,5.0,1387670400
0,0001388703,AC2PL52NKPL29,5.0,1378857600
1,0001388703,A1SUZXBDZSDQ3A,5.0,1362182400
2,0001388703,A3A0W7FZXM0IZW,5.0,1354406400
3,0001388703,A12R54MKO17TW0,5.0,1325894400
4,0001388703,A25ZT87OMIPLNX,5.0,1247011200


In [3]:
df.columns = ['BookID', 'ReviewerID', 'Rating', 'UnixReviewTime']
df

,BookID,ReviewerID,Rating,UnixReviewTime
0,0001388703,AC2PL52NKPL29,5.0,1378857600
1,0001388703,A1SUZXBDZSDQ3A,5.0,1362182400
2,0001388703,A3A0W7FZXM0IZW,5.0,1354406400
3,0001388703,A12R54MKO17TW0,5.0,1325894400
4,0001388703,A25ZT87OMIPLNX,5.0,1247011200
...,...,...,...,...
495,5557585400,A3FZMR0ABYPIMN,5.0,1470441600
496,5557585400,ADVHE3TVJ85UP,5.0,1468627200
497,5557585400,A1WX3Y3UVJXZUR,5.0,1468627200
498,5557585400,A3T6OFC2Q7O3SU,5.0,1465344000


In [4]:
# #Encode columns with categorical variables.
# s = (df.dtypes == 'object')
# object_cols = list(s[s].index)

# print(f'Categorical variables are:{object_cols}')

In [5]:
# ordinal_encoder = OrdinalEncoder()
# df[object_cols] = ordinal_encoder.fit_transform(df[object_cols])

In [8]:
combined = df.drop_duplicates(['ReviewerID', 'BookID'])
user_book_matrix = combined.pivot(index = 'ReviewerID', columns = 'BookID', values = 'Rating')
user_book_matrix.fillna(0, inplace = True)

user_book_matrix.head()

BookID,0001377647,0001388703,0001526146,0001527134,0006920055,0006935257,0615897398,0760103453,0760131694,0830838015,...,1937509095,278472414X,3236514752,3426958910,4901000616,5550312085,5552256646,5553860377,5557416978,5557585400
ReviewerID,,,,,,,,,,,,,,,,,,,,,
A106GSY0H5E2R4,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A106RJ1JBUA0TO,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10CSSGW3ESBCA,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10MO6SWEYJQ3F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11DP72VMLL5LS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [9]:
df['Rating'].value_counts()

5.0    428
4.0     32
1.0     19
3.0     15
2.0      6
Name: Rating, dtype: int64

In [10]:
#Build vocabularies to convert user IDs and book titles to integers for embedding layers.
user_ids_vocabulary = tf.keras.layers.StringLookup()
user_ids_vocabulary.adapt(df['ReviewerID'])

book_titles_vocabulary = tf.keras.layers.StringLookup()
book_titles_vocabulary.adapt(df['BookID'])

In [11]:
#Define a tfrs model.
class BookReviewModel(tfrs.Model):

  def __init__(
      self, user_model: tf.keras.Model, book_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
    super().__init__()

    #Set up user and book representations.
    self.user_model = user_model
    self.book_model = book_model

    #Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training = False):
    #Define how the loss is computed.
    user_embeddings = self.user_model(df['ReviewerID'])  
    book_embeddings = self.book_model(df['BookID'])

    return self.task(user_embeddings, book_embeddings)

In [12]:
#Convert 'book_id' to a tf.data object.

book_ID = tf.data.Dataset.from_tensor_slices(df['BookID'])
reviews = tf.data.Dataset.from_tensor_slices(df['ReviewerID'])

In [13]:
#Define the user and book models.
user_model = tf.keras.Sequential([
                                  user_ids_vocabulary,
                                  tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])

book_model = tf.keras.Sequential([
                                  book_titles_vocabulary,
                                  tf.keras.layers.Embedding(book_titles_vocabulary.vocab_size(), 64)
])

#Define your objectives.
task = tfrs.tasks.Retrieval(metrics = tfrs.metrics.FactorizedTopK(
    book_ID.batch(128).map(book_model)
))

In [14]:
#Create a retrieval model.
model = BookReviewModel(user_model, book_model, task)
model.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate = 0.5))



In [15]:
#Train for 3 epochs.
model.fit(reviews.batch(100), epochs = 3)

Epoch 1/3
5/5 [==============================] - 5s 75ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2036 - factorized_top_k/top_5_categorical_accuracy: 0.2196 - factorized_top_k/top_10_categorical_accuracy: 0.2396 - factorized_top_k/top_50_categorical_accuracy: 0.4580 - factorized_top_k/top_100_categorical_accuracy: 0.7288 - loss: 2180.4527 - regularization_loss: 0.0000e+00 - total_loss: 2180.4527
Epoch 2/3
5/5 [==============================] - 0s 77ms/step - factorized_top_k/top_1_categorical_accuracy: 0.3880 - factorized_top_k/top_5_categorical_accuracy: 0.4160 - factorized_top_k/top_10_categorical_accuracy: 0.4428 - factorized_top_k/top_50_categorical_accuracy: 0.6884 - factorized_top_k/top_100_categorical_accuracy: 0.9996 - loss: 1658.7821 - regularization_loss: 0.0000e+00 - total_loss: 1658.7821
Epoch 3/3
5/5 [==============================] - 0s 75ms/step - factorized_top_k/top_1_categorical_accuracy: 0.3604 - factorized_top_k/top_5_categorical_accuracy: 0.3896 - fact

In [16]:
#Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(
    model.user_model
)

index.index_from_dataset(
    reviews.batch(100).map(lambda title: (title, model.book_model(title)))
)

#Get some recommendations.
_, titles = index(np.array(['42']))

print(f'Top 5 recommendations for user 42 is:{titles[0, :3]}')

Top 5 recommendations for user 42 is:[b'AC2PL52NKPL29' b'A1SUZXBDZSDQ3A' b'A3A0W7FZXM0IZW']
